In [2]:
import pandas as pd
import os
import datetime as dt
import spmfunctions.spmfunctions as spm
import warnings
warnings.filterwarnings('ignore')

from scipy.stats import ttest_rel

In [3]:
intx='StLuke'
f_pkl=''#'df_raw_2401.pk'
yymms=['2312']#['2304','2305','2306','2307','2308','2309','2310','2311','2312']

path_pkl = '.\\ACHD_Eagle-{}\\df_raw'.format(intx)

if f_pkl:
    df_raw = pd.read_pickle(os.path.join(path_pkl,f_pkl))
elif yymms:
    df_raw = pd.concat([pd.read_pickle(os.path.join(path_pkl,f_pkl)) for f_pkl in os.listdir(path_pkl) if f_pkl[-7:-3] in yymms])

In [42]:
dfb,dfr = preempt_summary(df_raw,60)

In [ ]:
b=288550
a=100
df_raw[~df_raw.Code.isin([81,82])].loc[b:b+a,:]

In [ ]:
df_raw[(df_raw.Code==150)&(df_raw.ID.isin([2,3,4]))].head(50)

In [ ]:
df_trans = df_pe[(df_pe.PE_prev>=1)
                  & ((df_pe.Transition.apply(lambda x: '2' in str(x)))
                     | (df_pe.Transition.apply(lambda x: '3' in str(x)))
                     | (df_pe.Transition.apply(lambda x: '4' in str(x))))
                  & (df_pe.CP_change.isna())]
print(len(df_trans))
df_trans

In [ ]:
df_pe[(df_pe.Total>0)&(df_pe.Coord_plan!=254)]

In [ ]:
# Check if 150,3 always "travels together" with 175 and 200

df__=df_raw.loc[((df_raw.Code==150)&(df_raw.ID.isin([2,3,4])))|(df_raw.Code.isin([175,200])),
                ['TS_start','Code','ID']]\
            .groupby(['TS_start','Code'])\
            .apply(lambda x: str(list(x.ID))[1:-1].replace(' ',''))\
            .unstack('Code')


#df__.columns=df__.columns.droplevel(0)
df__.loc[(df__.loc[:,150].isin(['3','2']))].head(50)#&(df__.loc[:,175]!=8)].head(50)


In [ ]:
df_raw.loc[(df_raw.Code==152),['Coord_plan','ID']].drop_duplicates()

In [ ]:
df_raw=df_raw.drop(['R1','R2'],axis=1)

In [ ]:
# Add Ring columns

rings = {'R1':[1,2,3,4],
         'R2':[5,6,7,8]}

for r in rings.keys():
    dfr = df_raw.loc[(df_raw.Code==1),['Cycle_start','Coord_plan','ID']]\
                    .groupby(['Coord_plan','Cycle_start'])\
                    .apply(lambda x: [p for p in x.ID.to_list() if p in rings[r]])
    dfr.name = r
    df_raw = df_raw.join(dfr,on=['Coord_plan','Cycle_start'], how='left')
    
seq1 = df_raw[(df_raw.Coord_plan!=254)].groupby('Coord_plan').apply(lambda x: x.loc[:,'R1'].mode())
seq2 = df_raw[(df_raw.Coord_plan!=254)].groupby('Coord_plan').apply(lambda x: x.loc[:,'R2'].mode())

In [ ]:
def new_sort(l):
    m=l.copy()
    m.sort()
    return m

In [5]:
# Generate dataframe with coord phase for each cycle.
df_m = df_raw[df_raw.Code==152].groupby('Cycle_start',as_index=False).ID.min()
df_m = df_m.rename({'ID':'Coord_Ph'},axis=1)
df_m = df_m.set_index('Cycle_start')

# Add split column
for cp in df_m.Coord_Ph.unique():
    df_split = df_raw.loc[df_raw.Code==(133+cp),['Cycle_start','ID']]
    df_split = df_split.set_index('Cycle_start')
    df_split = df_split.rename({'ID':'Split'},axis=1)
    df_m = df_m.merge(df_split, how='outer',left_index=True,right_index=True, suffixes=['',cp])

df_m = df_m.ffill().dropna()

if len(df_m.Coord_Ph.unique())==2:
    df_m.loc[df_m.Coord_Ph==cp, 'Split'] = df_m.loc[df_m.Coord_Ph==cp, f'Split{cp}']
    df_m = df_m.drop(f'Split{cp}', axis=1)

# Add green start and end time columns
for cp in df_m.Coord_Ph.unique():
    df_be=df_raw[df_raw.Code.isin([1,10])
                 &(df_raw.ID==cp)
                 &(df_raw.Cycle_start.isin(df_m.index))]
    df_be=df_be.rename({'TS_start':'Start'},axis=1).sort_values('Start')
    df_be.loc[:,'End']=df_be.loc[:,'Start'].shift(-1)
    df_be=df_be.loc[df_be.Code==1,['Cycle_start','Start','End']]
    df_be = df_be.set_index('Cycle_start')

    df_m = df_m.merge(df_be, how='outer',left_index=True,right_index=True, suffixes=['',cp])
    
if len(df_m.Coord_Ph.unique())==2:
    df_m.loc[df_m.Coord_Ph==cp, 'Start'] = df_m.loc[df_m.Coord_Ph==cp, f'Start{cp}']
    df_m.loc[df_m.Coord_Ph==cp, 'End'] = df_m.loc[df_m.Coord_Ph==cp, f'End{cp}']
    df_m = df_m.drop([f'Start{cp}', f'End{cp}'], axis=1)

# Add local zero column
df_z=df_raw[(df_raw.Code==150)&(df_raw.ID==5)]
df_z.loc[:,'C']=(df_z.TS_start.shift(-1)-df_z.TS_start).dt.total_seconds()
df_z = df_z.rename({'TS_start' : 'Zero'}, axis=1)

df_m=df_m.merge(df_z.loc[:,['Cycle_start','Zero','C','Coord_plan']],
              left_index=True,right_on='Cycle_start',how='left').reset_index(drop=True).dropna()

# Calculate start and end in seconds from local zero, add duration column
df_m.loc[:,'Start_s'] = (df_m.loc[:,'Start']-df_m.loc[:,'Zero']).dt.total_seconds()
df_m.loc[:,'End_s'] = (df_m.loc[:,'End']-df_m.loc[:,'Zero']).dt.total_seconds()
df_m.loc[:,'Duration'] = (df_m.loc[:,'End']-df_m.loc[:,'Start']).dt.total_seconds()

# Filter out false cycles (i.e. cycles that go through uncoordinated periods)
df_m = df_m[(df_m.Start_s.apply(abs)<df_m.C)&(df_m.End_s>0)&(df_m.C.apply(abs)<1000)]

# Clip end to yeild point and start to yeild point-split
df_m.loc[(df_m.Start_s<df_m.Split),'Start_clip'] = df_m[df_m.Start_s<df_m.Split].Start_s.clip(lower=0)
df_m.loc[(df_m.Start_s<df_m.Split),'End_clip'] = df_m[df_m.Start_s<df_m.Split].End_s.clip(upper=df_m.Split)
df_m=df_m.dropna()

# Add  actual cycle length column calculated from Cycle_start
df_c = df_m.loc[:,['Cycle_start','Coord_plan']].drop_duplicates()
df_c.loc[:,'C_cs'] = (df_c.Cycle_start.shift(-1)-df_c.Cycle_start).dt.total_seconds()
df_m = df_m.merge(df_c.loc[:,['Cycle_start','C_cs']],on=['Cycle_start'],how='left')

# Calculate duration of green during coord green band
df_m.loc[:,'G_band'] = (df_m.End_clip-df_m.Start_clip)
df_g = df_m.loc[:,['C_cs','Zero','Duration','G_band']].groupby('Zero').sum()
df_m = pd.merge(df_m.drop(['C_cs'],axis=1),df_g,left_on='Zero',right_index=True,suffixes=['_s','_pct'])
df_m.loc[:,'G_band_pct'] = (df_m.G_band_s/df_m.Split)
df_m.loc[:,'Duration_pct'] = (df_m.Duration_pct/df_m.Split)

# Add pre-empt columns
df_pe = preempt_by_cycle(df_raw,False,False)
df_m = pd.merge(df_m,df_pe,on=['Cycle_start','Coord_plan'],how='left')#.fillna(0)

In [6]:
df_m=df_m.fillna('')

In [14]:
for ch in df_m.Channels.unique():
    print(ch, df_m[df_m.Channels==ch].Duration_pct.mean())
    print(df_m[(df_m.Prev1==ch)&(df_m.Channels=='')].Duration_pct.mean())
    print(df_m[(df_m.Prev2==ch)&(df_m.Prev1=='')&(df_m.Channels=='')].Duration_pct.mean())

 1.2802829561105424
1.2845091427306223
1.2897425752048433
4 1.1045406186639826
1.2632814334346025
1.1862847384960733
5 0.8740220608984455
1.1853940713853814
1.1945470942171428
3 1.3173263600546687
1.1803852288937353
1.240738918956816
3,4 1.136616388942414
1.2717008404763888
1.0970178213644497
6 0.9900528133977221
1.3045178734657374
1.2328825650540856
4,5 0.8427308210166559
1.2634599156118143
1.2067655954506815
3,5 1.0990007382375218
1.2021078096740168
1.136915358657674
3,4,5 1.1250812142937157
0.9946902654867257
0.9946902654867257
3,4,6 1.1113924050632913
nan
nan


In [7]:
for ch in df_m.Channels.unique():
    print(ch, df_m[df_m.Channels==ch].G_band_pct.mean())
    print(df_m[(df_m.Prev1==ch)&(df_m.Channels=='')].G_band_pct.mean())
    print(df_m[(df_m.Prev2==ch)&(df_m.Prev1=='')&(df_m.Channels=='')].G_band_pct.mean())

 0.9903533177790493
0.9908512274196855
0.9909706957301027
4 0.6394351895132127
0.9911504992493666
0.9914456254384048
5 0.6990117807612216
0.969066907539973
0.9893489521273275
3 0.9933784115845157
0.9807605474314456
0.9862327540199703
3,4 0.7189253652015174
0.9835484489678947
0.992373763511159
6 0.8705022757288988
0.9926274925866596
0.9926901214569651
4,5 0.6245526130245982
0.9893273765202283
0.9906680987618527
3,5 0.8005711476411401
0.981507164352597
0.9932722583146478
3,4,5 0.7391583585377693
0.9946902654867257
0.9946902654867257
3,4,6 0.7556962025316456
nan
nan


In [8]:
df_m.groupby('Channels').G_band_pct.count()

Channels
         8344
3         277
3,4        21
3,4,5       3
3,4,6       2
3,5         8
4         291
4,5         6
5         125
6           7
Name: G_band_pct, dtype: int64

In [83]:
bin_len=60
df_ch=df_m.loc[:,['Cycle_start','Channels']]\
                .set_index('Cycle_start')\
                .groupby('Channels')\
                .resample(f'{bin_len}T').count()\
                .unstack(level='Channels')#.fillna(0).reset_index()

df_ch.columns=df_ch.columns.droplevel(0)

In [97]:
df_m.head(50)

,Coord_Ph,Split,Start,End,Cycle_start,Zero,C,Coord_plan,Start_s,End_s,...,PE5,PE6,Total,Channels,Prev1,Aft1,Prev2,Aft2,Prev3,Aft3
0,4.0,79.0,2023-12-01 06:20:03.800,2023-12-01 06:21:51.400,2023-12-01 06:19:23.000,2023-12-01 06:20:15.400,236.1,1.0,-11.6,96.0,...,,,,,,,,,,4
1,4.0,79.0,2023-12-01 06:23:06.700,2023-12-01 06:25:28.700,2023-12-01 06:21:52.200,2023-12-01 06:24:11.500,158.8,1.0,-64.8,77.2,...,,,,,,,,4,,
2,4.0,79.0,2023-12-01 06:26:23.500,2023-12-01 06:28:08.700,2023-12-01 06:25:29.500,2023-12-01 06:26:50.300,160.0,1.0,-26.8,78.4,...,,,,,,4,,,,
3,4.0,79.0,2023-12-01 06:29:16.700,2023-12-01 06:29:35.500,2023-12-01 06:28:09.500,2023-12-01 06:29:30.300,160.1,1.0,-13.6,5.2,...,,,1.0,4,,,,,,
4,4.0,79.0,2023-12-01 06:30:09.600,2023-12-01 06:30:51.600,2023-12-01 06:28:09.500,2023-12-01 06:29:30.300,160.1,1.0,39.3,81.3,...,,,1.0,4,,,,,,
5,4.0,79.0,2023-12-01 06:32:00.400,2023-12-01 06:33:28.400,2023-12-01 06:30:52.400,2023-12-01 06:32:10.400,160.0,1.0,-10.0,78.0,...,,,,,4,,,,,
6,4.0,79.0,2023-12-01 06:34:36.000,2023-12-01 06:36:08.400,2023-12-01 06:33:29.600,2023-12-01 06:34:50.400,160.0,1.0,-14.4,78.0,...,,,,,,,4,,,
7,4.0,79.0,2023-12-01 06:37:14.400,2023-12-01 06:38:48.400,2023-12-01 06:36:09.600,2023-12-01 06:37:30.400,160.0,1.0,-16.0,78.0,...,,,,,,,,,4,
8,4.0,79.0,2023-12-01 06:39:54.400,2023-12-01 06:41:28.400,2023-12-01 06:38:49.600,2023-12-01 06:40:10.400,160.0,1.0,-16.0,78.0,...,,,,,,,,,,
9,4.0,79.0,2023-12-01 06:42:29.200,2023-12-01 06:44:08.400,2023-12-01 06:41:29.600,2023-12-01 06:42:50.400,160.0,1.0,-21.2,78.0,...,,,,,,,,,,


In [95]:
bin_len=15
df_m.loc[:,['Cycle_start','Total']]\
                .set_index('Cycle_start')\
                .resample(f'{bin_len}T').sum().describe([0.75,0.8,0.85,0.9,0.95,0.99,0.995,0.999,0.9995])#.groupby('Total').count()

ValueError: Cannot describe a DataFrame without columns

In [44]:
df_m.set_index('Cycle_start').groupby('Channels').resample('60T')

Coord_Ph                                       Split  ...  \
Channels                         3 3,4 3,6    4 4,5 4,6   5    6          ...   
Cycle_start                                                               ...   
2023-09-01 06:00:00    120.0   NaN NaN NaN  NaN NaN NaN NaN  NaN  1125.0  ...   
2023-09-01 07:00:00    176.0   NaN NaN NaN  NaN NaN NaN NaN  NaN  1650.0  ...   
2023-09-01 08:00:00    176.0  24.0 NaN NaN  NaN NaN NaN NaN  NaN  1650.0  ...   
2023-09-01 09:00:00    168.0   0.0 NaN NaN  NaN NaN NaN NaN  NaN  1515.0  ...   
2023-09-01 10:00:00    184.0   0.0 NaN NaN  NaN NaN NaN NaN  NaN  1587.0  ...   
...                      ...   ...  ..  ..  ...  ..  ..  ..  ...     ...  ...   
2023-09-30 17:00:00     76.0   0.0 NaN NaN  8.0 NaN NaN NaN  4.0  1482.0  ...   
2023-09-30 18:00:00     84.0   0.0 NaN NaN  NaN NaN NaN NaN  NaN  1638.0  ...   
2023-09-30 19:00:00    100.0   0.0 NaN NaN  NaN NaN NaN NaN  NaN  1375.0  ...   
2023-09-30 20:00:00    104.0   0.0 NaN NaN  NaN NaN NaN NaN  NaN  1430.0  ...   
2023-09-30 21:00:00     92.0   4.0 NaN NaN  NaN NaN NaN NaN  NaN  1265.0  ...   

                    Duration_pct G_band_pct                                  \
Channels                       6                    3 3,4 3,6         4 4,5   
Cycle_start                                                                   
2023-09-01 06:00:00          NaN  14.718667       NaN NaN NaN       NaN NaN   
2023-09-01 07:00:00          NaN  21.502667       NaN NaN NaN       NaN NaN   
2023-09-01 08:00:00          NaN  21.528000  1.722667 NaN NaN       NaN NaN   
2023-09-01 09:00:00          NaN  20.704406  0.000000 NaN NaN       NaN NaN   
2023-09-01 10:00:00          NaN  22.568116  0.000000 NaN NaN       NaN NaN   
...                          ...        ...       ...  ..  ..       ...  ..   
2023-09-30 17:00:00     0.558974  18.480769  0.000000 NaN NaN  1.979487 NaN   
2023-09-30 18:00:00          NaN  20.637179  0.000000 NaN NaN       NaN NaN   
2023-09-30 19:00:00          NaN  24.492727  0.000000 NaN NaN       NaN NaN   
2023-09-30 20:00:00          NaN  25.332727  0.000000 NaN NaN       NaN NaN   
2023-09-30 21:00:00          NaN  22.409091  0.974545 NaN NaN       NaN NaN   

                                       
Channels            4,6   5         6  
Cycle_start                            
2023-09-01 06:00:00 NaN NaN       NaN  
2023-09-01 07:00:00 NaN NaN       NaN  
2023-09-01 08:00:00 NaN NaN       NaN  
2023-09-01 09:00:00 NaN NaN       NaN  
2023-09-01 10:00:00 NaN NaN       NaN  
...                  ..  ..       ...  
2023-09-30 17:00:00 NaN NaN  0.558974  
2023-09-30 18:00:00 NaN NaN       NaN  
2023-09-30 19:00:00 NaN NaN       NaN  
2023-09-30 20:00:00 NaN NaN       NaN  
2023-09-30 21:00:00 NaN NaN       NaN  

[712 rows x 117 columns]

In [ ]:
for p in [4,8]:
    print('Phase ',p)
    df_n = df_m.copy()[(df_m.Total.isna())
                      &(df_m.Coord_Ph==p)
                      &(df_m.PE_prev.isna())]
    
    df_n = df_n.groupby('Coord_plan').G_band_pct.mean()
    df_n.name = 'No_PE_' + df_n.name

    df_ = df_m.copy()[(df_m.PE_prev==1)&(df_m.Coord_Ph==p)]
    df_ = df_.groupby('Coord_plan').G_band_pct.mean()
    df_.name = 'Prev_' + df_.name
    
    print(pd.concat([df_n, df_],axis=1))
    #sort_values('G_band_pct').head(50)

In [ ]:
for p in [4,8]:
    print('Phase ',p)
    df_n = df_m.copy()[(df_m.Total.isna())
                      &(df_m.Coord_Ph==p)
                      &(df_m.PE_prev.isna())]
    df_n = df_n.groupby('Coord_plan').G_band_pct.mean()
    df_n.name = 'No_PE_' + df_n.name

    for pe in [3,4,5,6]:
        df_=df_m.copy()[(df_m.loc[:,f'PE{pe}']==1)&(df_m.Total==1)&(df_m.Coord_Ph==p)]
        df_=df_.groupby('Coord_plan').G_band_pct.mean()
        df_.name = f'PE{pe}_' + df_.name

        print(pd.concat([df_n, df_],axis=1))

    #sort_values('G_band_pct').head(50)

In [ ]:
# Check where coord phase changes

df_min=df_raw.loc[df_raw.Code==152,['Cycle_start','ID','Coord_plan']].groupby('Cycle_start').min().reset_index(drop=False)
df_max=df_raw.loc[df_raw.Code==152,['Cycle_start','ID','Coord_plan']].groupby('Cycle_start').max().reset_index(drop=False)
df_coord_ph=pd.merge(df_min,df_max,on=['Cycle_start','Coord_plan'])
df_coord_ph.loc[:,'Prev']=df_coord_ph.groupby('Coord_plan').ID_x.shift(1)
df_coord_ph[df_coord_ph.ID_x!=df_coord_ph.Prev].head(50)

In [ ]:
b=315
df_m.loc[b:b+20,:]

In [ ]:
df=df_m[df_m.Coord_plan==1].reset_index(drop=True)
plot_multiple_ts(df, 'g band.html')

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_multiple_ts(df, save_path, title = '', y_lab = ''):
    
    fig = make_subplots(rows=1, cols=1, subplot_titles=[title])
    
    df1=df[]
    
    for i, row in df.iterrows():
        fig.add_trace(px.line(x=[row.StartS,row.EndS], y=[i,i]).data[0])

    #fig.update_layout(xaxis=dict(title='Time', type="date"),
    #                  yaxis=dict(title=y_lab))

    fig.add_trace(px.line(x=[0,0],y=[0,len(df)]).data[0])
    fig.add_trace(px.line(x=df.loc[0:1,'Split'].to_list(),y=[0,len(df)]).data[0])
    fig.update_layout(showlegend=False)
    #fig.update_layout(title_text=f"{intx} Phase Termination")
    fig.layout.xaxis.autorange = True
    fig.layout.yaxis.autorange = True
    
    fig.write_html(save_path)    

In [ ]:
i=5700
df_m.loc[i:i+20]

In [ ]:
print(dt.datetime.now())
ints=['I84EB','I84WB','StLuke','Franklin','Pine',
     'Fairview','RiverValley','Ustick','Wainwright',
     'McMillan','HobbleCreek','Chinden',
     'Colchester','IslandWoods', 'Riverside'
     ]
sfx='_2304-2312__'
yymms=['2304','2305','2306','2307','2308','2309','2310','2311','2312']
resdict={}

with pd.ExcelWriter('./EagleRd EVPE_Speed%s.xlsx' % (sfx),
                    mode='w', engine='openpyxl') as writer:
    i=0
    for intx in ints:
        path_pkl = '.\\ACHD_Eagle-{}\\df_raw'.format(intx)
        df_raw = pd.concat([pd.read_pickle(os.path.join(path_pkl,f_pkl)) for f_pkl in os.listdir(path_pkl) if f_pkl[-7:-3] in yymms])
        df_b,df_m,df_ba=pe_before_after_speed(df_raw,intx)
        df_ba.loc[:,'Intersection']=intx
        if i==0:
            resdf=df_ba.copy()
            i+=1
        else:
            resdf=resdf.append(df_ba)
        #df_bin,df_pe=pe_sum(df_raw, 5)
        #df_pe.to_excel(writer, sheet_name=intx)
        
        df_b.to_excel(writer, sheet_name=f'{intx}_PE_speed')
        df_m.to_excel(writer, sheet_name=f'{intx}_PE_code')
        #resdict[intx]=(df_bin, df_pe)
    
    resdf.index=pd.MultiIndex.from_frame(resdf.reset_index().loc[:,['Intersection','PE']])
    resdf=resdf.drop('Intersection',axis=1)
    resdf.to_excel(writer, sheet_name='Stat_Summary')
print(dt.datetime.now())

In [ ]:
intx='Franklin'
yymms=['2312','2311','2310','2309']

df_raw=pd.concat([pd.read_pickle(f'./ACHD_Eagle-{intx}/df_raw/df_raw_{yymm}.pk') for yymm in yymms]).sort_values('TS_start').reset_index(drop=True)

df_tt=read_inrix(5,'pk',intx,True)

dfbin,dfres=pe_sum(df_raw,5)
dfbin.loc[:,'Minute']=dfbin.TS_start.dt.minute
dfbin.loc[:,'Day']=dfbin.TS_start.dt.day_name()

df_mean=df_tt.groupby([df_tt.index.day_name(), df_tt.index.hour, df_tt.index.minute]).mean()
df_mean.index=df_mean.index.set_names(['Day','Hour','Minute'])
df_mean=df_mean.reset_index()

df_btt=pd.merge(dfbin,df_tt,left_on='TS_start',right_on='Date_Time',how='left')
df_btt=pd.merge(df_btt,df_mean,on=['Day','Hour','Minute'],suffixes=['','_mean']).sort_values('TS_start')

In [ ]:
dirs=[c.split('_')[0] for c in df_tt.columns if 'mph' in c]
for d in dirs:
    df_btt.loc[:,f'{d}_mean_diff']=df_btt.loc[:,f'{d}_mph']-df_btt.loc[:,f'{d}_mph_mean']
    df_btt.loc[:,f'{d}_next_diff']=df_btt.loc[:,f'{d}_mph']-df_btt.loc[:,f'{d}_mph'].shift(1)
    print(f'{d} mean diff: ', df_btt[f'{d}_mean_diff'].apply(abs).mean())
    print(f'{d} next diff: ', df_btt[f'{d}_next_diff'].apply(abs).mean())

In [ ]:
'''
102 Preempt Call Input On
104 Preempt Call Input Off
105 Preempt Entry Started
107 Preempt Begin Dwell
111 Preempt Begin Exit Interval
116 Preempt Events Reserved For Future Use

102->105 Blue (Delay Start, typically 0 or 0.4s)
105->107 Orange (Time to Service)
107->111(or 104) Green (Dwell Time) 

PE3-7&4
PE4-3&8
PE5-2&5
PE6-1&6

150 Coord State:
0=Free
1=In Step
2=Transition-Add
3=Transition-Subtract
4=Transition-Dwell
5=Local Zero
6=Begin Pickup
7=Master Cycle Zero
'''


#.groupby('Code').count()

In [ ]:
df_bin,dfr=pe_sum(df_raw,5)
df_bin.loc[:,'Total']=df_bin.iloc[:,1:5].sum(axis=1)
df_bin=df_bin.set_index('TS_start')
df_bin.loc[:,'Hour_Total']=df_bin.Total.rolling(12).sum()
df_bin=df_bin.sort_values(['Hour_Total','Total'],ascending=False)
df_bin.loc[:,'Rank']=list(range(1,len(df_bin)+1))
df_bin=df_bin.sort_index().reset_index(drop=False)

In [ ]:
exc=[]
for i,row in df_bin.sort_values('Rank').iterrows():
    if i in exc:
        continue
    df_bin.loc[i-11:i-1,['Hour_Total','Rank']]=0
    df_bin.loc[i+1:i+12,['Hour_Total','Rank']]=0
    exc+=list(range(i-11,i+12))

In [ ]:
df_bin[df_bin.Rank>0].sort_values('Rank').to_csv('PE_rank.csv')

In [ ]:
df_gyr=spm.comb_gyr_det(spm.filter_date(df_raw,'12-1-2023','12-30-2023','TS_start'))
df_gyr=spm.add_r1r2(df_gyr)

In [ ]:
df_g=df_gyr[(df_gyr.Code==1)]
df_cycles=df_g.loc[:,['Cycle_start','Coord_plan','Cycle_len','R1_Ph','R2_Ph']].drop_duplicates()
df_cycles=df_cycles.set_index('Cycle_start')

df_g=df_g.loc[:,['Cycle_start','ID','Split']].groupby(['Cycle_start','ID']).sum().unstack('ID')
df_g.columns=[f'Ph{c}' for c in df_g.columns.droplevel(0)]

df_cycles=pd.merge(df_cycles, df_g, left_index=True, right_index=True)

df_pe=df_raw[df_raw.Code==105].groupby(['Cycle_start','ID']).count().Code.unstack('ID').fillna(0)
df_pe.columns=[f'PE{c}' for c in df_pe.columns]
df_pe.loc[:,'Total']=df_pe.sum(axis=1)

dfm=pd.merge(df_cycles,df_pe,left_index=True,right_index=True,how='left')

In [ ]:
dfm.to_csv(f'{intx}_EVPE_Splits_.csv')

In [ ]:
print(len(df_g))
print(len(df_cycles))

In [ ]:
# Produce tabular split data joined with pre-empt counts on cycle-by-cycle basis

df_gyr=spm.comb_gyr_det(df_raw)
df_gyr=spm.add_r1r2(df_gyr)

df__=df_gyr[(df_gyr.Code==1)]
df_cycles=df__.loc[:,['Cycle_start','Coord_plan','Cycle_len','R1_Ph','R2_Ph']].drop_duplicates()
df_cycles=df_cycles.set_index('Cycle_start')

df_no_rpt=df_cycles[df_cycles.R1_Ph.apply(lambda x: len(list(x))==len(set(x)))&
                    df_cycles.R2_Ph.apply(lambda x: len(list(x))==len(set(x)))]

for i,row in df_no_rpt.iloc[:,-2:].drop_duplicates().iterrows():
    df=df__[(df__.R1_Ph==row.R1_Ph)&(df__.R2_Ph==row.R2_Ph)]
    #df=df.pivot('Cycle_start','ID','Split')
    df=df.groupby('')
    df_cycles.loc[df.index,df.columns]=df
    
d=dict()
for i, gb in df_cycles.groupby('Coord_plan'):
    d[i]=gb.describe([0.1,0.9])
    
cols=[x for x in df_cycles.columns if '_' not in str(x)]
cols.sort()
cols=['Cycle_len']+cols
df_c_floor=df_cycles.copy()
df_c_ceiling=df_cycles.copy()
for i in d.keys():
    df_c_floor.loc[df_cycles.Coord_plan==i,cols] = df_cycles>=d[i].loc['10%',cols]
    df_c_ceiling.loc[df_cycles.Coord_plan==i,cols] = df_cycles<=d[i].loc['90%',cols]
    
df_pe=df_raw[df_raw.Code==105].groupby(['Cycle_start','ID']).count().Code.unstack('ID').fillna(0)
df_pe.columns=[f'PE{c}' for c in df_pe.columns]
df_pe.loc[:,'Total']=df_pe.sum(axis=1)


dfm=pd.merge(df_cycles,df_pe,left_index=True,right_index=True,how='left')

dfm.to_csv(f'{intx}_EVPE_Splits.csv')

In [34]:
def preempt_by_cycle(df_raw, add_ped=True, add_transition=True):

    # Count pre-empt entry events per cycle
    df_pe_ct = df_raw[df_raw.Code==105].groupby(['Cycle_start','ID']).count()
    df_pe = df_pe_ct.Code.unstack('ID')
    df_pe.columns = [f'PE{c}' for c in df_pe.columns]
    df_pe.loc[:, 'Total'] = df_pe.sum(axis=1)
    df_pe.loc[:, 'Channels'] = df_pe_ct.reset_index(drop=False)\
                                       .groupby('Cycle_start')\
                                       .apply(lambda x: str(list(x.ID))[1:-1].replace(' ',''))
    
    # Add back all cycles
    all_cyc=df_raw.loc[:,['Cycle_start','Coord_plan']].drop_duplicates()
    df_pe = pd.merge(all_cyc, df_pe, left_on='Cycle_start', right_index=True, how='left')

    # Add marker for cycles within 3 cycles before and after PE event
    df_pe = df_pe.reset_index(drop=True)
    idx = df_pe[df_pe.Total>0].index
    for i in [1,2,3]:
        df_pe.loc[:,f'Prev{i}'] = df_pe.loc[:,'Channels'].shift(i)
        df_pe.loc[:,f'Aft{i}'] = df_pe.loc[:,'Channels'].shift(-i)
    
    # Add ped call counts
    if add_ped:
        df_ped = spm.ped_counts(df_raw,'cycle')
        df_pe = pd.merge(df_pe, df_ped, left_on='Cycle_start', right_index=True, how='left')

    # Add column with list of each transition code
    if add_transition:
        df_tr=df_raw.loc[((df_raw.Code==150)),
                        ['Cycle_start','ID']]\
                    .groupby('Cycle_start')\
                    .apply(lambda x: str(list(x.ID))[1:-1].replace(' ',''))
        df_tr.name='Transition'
        df_pe = pd.merge(df_pe, df_tr, left_on='Cycle_start', right_index=True, how='left')


        # Add marker for cycles before and after changing coord plan
        df_pe.loc[df_pe.Coord_plan!=df_pe.Coord_plan.shift(), 'CP_change'] = 1
        for s in [-1,-1,1,1]:
            df_pe.loc[df_pe.CP_change.shift(s)==1,'CP_change'] = 1
        
    return df_pe


def preempt_summary(df_raw, bin_len):    
    pe_key={3:'SB',4:'NB',5:'WB',6:'EB', '':'Total'}
    day_key={0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun','Weekday':'Weekday','Weekend':'Weekend'}
    
    df_bin=df_raw.loc[(df_raw.Code==105),['TS_start','ID']]\
                .set_index('TS_start')\
                .groupby('ID')\
                .resample(f'{bin_len}T').count()\
                .unstack(level='ID').fillna(0).reset_index()
    df_bin.loc[:,'Total']=df_bin.iloc[:,1:].sum(axis=1)
    df_bin.loc[:,'Weekday']=df_bin.TS_start.dt.day_of_week

    df_bin.loc[:,'Hour']=df_bin.TS_start.dt.hour
    df_byday=df_bin.groupby(['Weekday','Hour']).mean().unstack(level='Weekday')
    df_bin.loc[df_bin.Weekday<5,'Weekday']='Weekday'
    df_bin.loc[df_bin.Weekday.isin([5,6]),'Weekday']='Weekend'    

    df_wday_wknd=df_bin.groupby(['Weekday','Hour']).mean().unstack(level='Weekday')
    df_res=pd.merge(df_byday,df_wday_wknd,left_index=True,right_index=True)
    df_res.columns=df_res.columns.droplevel(0)
    
    df_bin.columns=[(str(x)+str(y)).strip('ID') for x,y in df_bin.columns]    
    df_res.index.name=None

    df_res.columns=[f'PE{pe}_{pe_key[pe]}_{day_key[day]}' for pe, day in df_res.columns]
    df_bin=df_bin.rename(dict([(pe,f'PE{pe}_{pe_key[int(pe)]}') for pe in df_bin.columns[1:-3]]),axis=1)
    
    return df_bin,df_res


def read_inrix(bin_len, f_type, intx=None, pivot=False):
    
    # Read and format segment ID key
    if intx:
        df_key=pd.read_csv('./Inrix/INRIX_key.csv')
        df_key=df_key[df_key.Intersection==intx]
    
    # Read and format csv
    if f_type=='csv':
        df_tt=pd.read_csv(f'./Inrix/data_{bin_len}.csv')
        df_tt.columns=[x.replace(' ','_') for x in df_tt.columns]
        df_tt=df_tt[df_tt.CValue>0]
        if intx:
            df_tt=df_tt[df_tt.Segment_ID.isin(df_key.loc[:,'Segment ID'])]
        df_tt.Date_Time = pd.to_datetime(df_tt.Date_Time).apply(
            lambda x: x.replace(tzinfo=None))    

    # Or read from pickle (pre-formatted)
    elif f_type=='pk':
        df_tt=pd.read_pickle(f'./Inrix/data_{bin_len}.pk')
        
        #Filter to intersection
        if intx:
            df_tt=df_tt[df_tt.Segment_ID.isin(df_key.loc[:,'Segment ID'])]
    
    # Pivot and re-name columns
    if pivot:
        df_tt=df_tt.loc[:,['Date_Time','Segment_ID','Speed(miles/hour)']]\
                   .pivot(index='Date_Time', columns='Segment_ID', values='Speed(miles/hour)')
        df_key=df_key.set_index('Segment ID')    
        df_tt.columns=[f'{df_key.loc[seg,"Dir"]}_mph' for seg in df_tt.columns]       

    return df_tt

def pe_before_after_speed(df_raw, intx):

    df_tt=read_inrix(5, 'pk', intx, pivot=True)
    df_bin,df_pe=pe_sum(df_raw,5)
    df_bin=df_bin.drop(['Weekday','Hour'],axis=1)

    df_bin.loc[:,'Same']=df_bin.iloc[:,1:].sum(axis=1).clip(upper=1)
    #df_bin.loc[:,'Before']=df_bin.Same.shift(1)
    df_bin.loc[:,'After']=df_bin.Same.shift()
    df_bin.loc[:,'Activity']=df_bin.loc[:,['Same','After']].sum(axis=1).clip(upper=1)
    df_bin=df_bin.drop(['Same','After'],axis=1)

    df_bin.loc[:,'Act_shift']=df_bin.Activity.shift(-1)
    df_bin.loc[df_bin.Activity==0,'Event']=0
    df_bin.loc[((df_bin.Activity==0)&(df_bin.Act_shift==1)),'Event']=df_bin.loc[((df_bin.Activity==0)&(df_bin.Act_shift==1)),:].index
    df_bin=df_bin.drop(['Activity','Act_shift'],axis=1)
    df_bin.loc[:,'Event']=df_bin.Event.ffill()
    df_bin=df_bin[df_bin.Event>0]
    
    rep_dict={}
    for i,j in pd.Series(df_bin.Event.unique()).iteritems():
        rep_dict[j]=i+1
    df_bin.loc[:,'Event']=df_bin.Event.replace(rep_dict)
    
    pe_cols=df_bin.columns[1:-1].to_list()
    
    # Change df_mgb line below if want merge after groupby
    df_bin=pd.merge(df_bin[df_bin.Event>0], df_tt, 'left', left_on='TS_start', right_on='Date_Time')
    
    spd_cols=df_tt.columns.to_list()
    b_cols=[c+'_Bef' for c in spd_cols]
    a_cols=[c+'_Aft' for c in spd_cols]
    
    df_m=pd.DataFrame(columns=pe_cols+['PE']+b_cols+a_cols)

    for t, dfgb in df_bin.groupby('Event'):

        srs=pd.Series(index=pe_cols+['PE', 'Duration']+b_cols+a_cols)
#        if t==0:
#            continue
        df_mgb=dfgb.reset_index(drop=True)#pd.merge(dfgb, df_tt, 'left', left_on='TS_start', right_on='Date_Time')

        srs.name=df_mgb.iloc[1,0]
        srs['Duration']=(len(df_mgb)-1)*5
        #srs['TS_start']=df_m.iloc[1,0]
        srs[pe_cols]=df_mgb[pe_cols].sum()#mean()
        srs[a_cols]=df_mgb.loc[1:,spd_cols].mean().values
        srs[b_cols]=df_mgb.loc[0,spd_cols].values

        if len(dfgb)<4: # SINGLE TIME
            if srs[pe_cols].sum()==srs[pe_cols].max(): # SINGLE PE
                srs['PE']=int(srs[pe_cols][srs[pe_cols]>0].index[0][2])
            else: # MULTIPLE PES
                srs['PE']=9
        else: # MULTIPLE TIMES
            if srs[pe_cols].sum()==srs[pe_cols].max(): # SINGLE PE
                srs['PE']=int(srs[pe_cols][srs[pe_cols]>0].index[0][2])*11
            else: # MULTIPLE PES    
                srs['PE']=99
        
        df_m=df_m.append(srs)        

    df_bin=df_bin.set_index('TS_start')
    df_bin.index.name='Time'

    spd_cols=[c for c in df_m.columns if 'mph' in c]
    dirs=[c.split('_')[0] for c in df_m.columns if 'mph_Bef' in c]
    
    resdf=df_m.loc[:,['PE']+spd_cols].groupby('PE').mean()
    
    for pe,dfgb in df_m.loc[:,['PE']+spd_cols].groupby('PE'):
        for d in dirs:
            t_stat,pval=ttest_rel(dfgb.loc[:,f'{d}_mph_Bef'],dfgb.loc[:,f'{d}_mph_Aft'],nan_policy='omit')
            resdf.loc[pe,f'{d}_t-stat']=t_stat
            resdf.loc[pe,f'{d}_p-value']=pval
            resdf.loc[pe,f'{d}_N']=len(dfgb.loc[:,[f'{d}_mph_Bef',f'{d}_mph_Aft']].dropna())    

    cols=[]
    ms=['_mph_Bef','_mph_Aft','_diff','_N','_t-stat','_p-value']

    for d in dirs:
        cols+=[d+m for m in ms]
        resdf.loc[:,f'{d}_diff']=resdf.loc[:,f'{d}_mph_Aft']-resdf.loc[:,f'{d}_mph_Bef']
    resdf=resdf[cols]
    return df_bin, df_m, resdf    

In [ ]:
pe_cols=df_pe.columns[1:]
df_pe.loc[:,pe_cols]=df_pe.loc[:,pe_cols].clip(upper=1)

df_merge=pd.merge(df_pe,df_tt,how='left',left_on='TS_start',right_on='Date_Time')
df_merge.loc[:,'Current']=df_merge.loc[:,pe_cols].sum(axis=1)
df_merge.loc[:,'Before']=df_merge.Current.shift(-1)
df_merge.loc[:,'After']=df_merge.Current.shift()
df_merge.loc[df_merge.After==1,'TS_start']=df_merge.TS_start.shift()[df_merge.After==1]
df_merge.loc[df_merge.Before==1,'TS_start']=df_merge.TS_start.shift(-1)[df_merge.Before==1]
df_merge.columns=[str(c) for c in df_merge.columns]

segs=[str(c) for c in df_merge.columns[2:-3] if int(c)>10]
cols=['TS_start']+segs
df_m=pd.merge(df_merge.loc[(df_merge.Current==1)&(df_merge.Before==0)&(df_merge.After==0),df_merge.columns[:-3]],
              df_merge.loc[(df_merge.Current==0)&(df_merge.Before==1)&(df_merge.After==0),cols],
              how='left',on='TS_start',suffixes=('','_Before')
             )
df_m=pd.merge(df_m,
              df_merge.loc[(df_merge.Current==0)&(df_merge.Before==0)&(df_merge.After==1),cols],
              how='left',on='TS_start',suffixes=('','_After')
             )

for seg in segs:
    df_m.loc[:,f'D1_{seg}']=df_m[seg]-df_m[f'{seg}_Before']
    df_m.loc[:,f'D2_{seg}']=df_m[f'{seg}_After']-df_m[f'{seg}_Before']
    


In [ ]:
df_m.to_csv('Chinden_PE_TT.csv')

In [ ]:
# Before/After green time

# Define dataframe df

#df=spm.filter_date(df_raw,'11-29-2023','11-29-2023','TS_start').reset_index(drop=True)
df=df_raw.copy()

# Create before/after indices
df_cs=df.loc[:,['Cycle_start']].drop_duplicates().reset_index(drop=True)
df_cs.loc[:,'C']=(df_cs.Cycle_start.shift(-1)-df_cs.Cycle_start).dt.total_seconds()
df_pe=df.loc[df.Code==105,['Cycle_start','ID','Coord_plan']]
idx=df_cs[df_cs.Cycle_start.isin(df_pe.Cycle_start.values)].index
idx_aft=idx.to_list()+(idx+1).to_list()+(idx+2).to_list()
idx_bef=(idx-1).to_list()+(idx-2).to_list()+(idx-3).to_list()

# Process After dataframe

df_aft=df[(df.Cycle_start.isin(df_cs.loc[idx_aft].Cycle_start.values))]
df_aft.loc[df_aft.Code.isin([1,8]),'Duration']=(df_aft[df_aft.Code.isin([1,8])].groupby('ID').TS_start.shift(-1)-\
                                               df_aft.loc[df_aft.Code.isin([1,8]),'TS_start']).dt.total_seconds()
df_aft=df_aft.loc[df_aft.Code==1,['Cycle_start','ID','Duration']].groupby(['Cycle_start','ID']).sum().unstack().fillna(0)
df_aft.columns=df_aft.columns.droplevel(0)
df_aft.loc[:,'BA']='After'
df_aft=df_aft.reset_index()

df_aft=df_aft.merge(df_pe.drop_duplicates(),left_on='Cycle_start',right_on='Cycle_start', how='outer').sort_values('Cycle_start')
df_aft.loc[:,'ID']=df_aft.ID.ffill()
df_aft.loc[:,'Coord_plan']=df_aft.Coord_plan.ffill()

# Process Before dataframe

df_bef=df[(df.Cycle_start.isin(df_cs.loc[idx_bef].Cycle_start.values))]
df_bef.loc[df_bef.Code.isin([1,8]),'Duration']=(df_bef[df_bef.Code.isin([1,8])].groupby('ID').TS_start.shift(-1)-\
                                               df_bef.loc[df_bef.Code.isin([1,8]),'TS_start']).dt.total_seconds()
df_bef=df_bef.loc[df_bef.Code==1,['Cycle_start','ID','Duration']].groupby(['Cycle_start','ID']).sum().unstack().fillna(0)
df_bef.columns=df_bef.columns.droplevel(0)
df_bef.loc[:,'BA']='Before'
df_bef=df_bef.reset_index()

df_bef=df_bef.merge(df_pe.drop_duplicates(),left_on='Cycle_start',right_on='Cycle_start', how='outer').sort_values('Cycle_start')
df_bef.loc[:,'ID']=df_bef.ID.bfill()
df_bef.loc[:,'Coord_plan']=df_bef.Coord_plan.bfill()

# Combine before and after dataframes

df_ba=pd.concat([df_bef,df_aft]).sort_values('Cycle_start').dropna().reset_index(drop=True)
df_ba=df_ba.merge(df_cs,left_on='Cycle_start',right_on='Cycle_start')
df_ba.loc[:,1:8]=df_ba.loc[:,1:8].apply(lambda x: x/df_ba.C)

In [ ]:
df_ba.groupby(['ID','Coord_plan','BA']).mean().to_csv('Fairview_2311_EVPE_green_BA.csv')

In [ ]:
df_ba.groupby('BA').mean()